In [1]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))

env = Environment(
    method="LBL",
    size=3,
    args=args["environment"]
)

agent = DQN(env, args["DQN"])

In [5]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [ ]:
env.algorithm.status(cube)

front LLLLLLLLL RRRRRRRRR
right FFFFFFFFF BBBBBBBBB
back RRRRRRRRR LLLLLLLLL
left BBBBBBBBB FFFFFFFFF


100

: 

In [9]:
env.scramble()
print(env.state2)
print(env.state2.shape)

[[[ 4  2  1  0  6]
  [ 2  1 -1  1  7]
  [ 4  2  0  2 23]
  [ 4 -1  2  3 14]
  [ 1 -1 -1  4  4]
  [ 4 -1  0  5 20]
  [ 0  3  4  6 17]
  [ 0  5 -1  7 22]
  [ 0  5  2  8 25]
  [-1  3  5  9 11]
  [-1  3 -1 10 10]
  [-1  2  5 11 16]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  5  1 14  5]
  [-1  2 -1 15 15]
  [-1  3  1 16  1]
  [ 1  3  5 17  2]
  [ 0  3 -1 18 18]
  [ 0  3  5 19 19]
  [ 4 -1  3 20  9]
  [ 0 -1 -1 21 21]
  [ 0 -1  2 22 24]
  [ 2  5  1 23  8]
  [ 1  4 -1 24  3]
  [ 4  3  1 25  0]]]
(1, 26, 5)


In [10]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 2, 12)
)


In [11]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[ 0.0287, -0.2069, -0.4253, -0.0753,  0.2927,  0.0515,  0.2215,  0.3008,
          0.1258, -0.0102, -0.2572,  0.1653]], grad_fn=<AddmmBackward0>)

In [12]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[  2.0019,   1.7520,  -1.0355,  ...,  -2.4831,  -0.3922,   1.4125],
        [  4.8899,   4.5193,  -2.0768,  ...,  -5.9477,  -1.5034,   3.6079],
        [  7.7780,   7.2866,  -3.1181,  ...,  -9.4123,  -2.6145,   5.8033],
        [ 10.6660,  10.0539,  -4.1593,  ..., -12.8769,  -3.7257,   7.9987]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[0.0911, 1.2843],
        [0.9483, 2.6599],
        [1.8056, 4.0356],
        [2.6628, 5.4112]], grad_fn=<AddmmBackward0>)
